In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
# Helper libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import files
import io

In [2]:
# load mostly preprocessed data (preprocessing accomplished before 3/25 proposal)

# would use regular numpy syntax to load locally:
#X = np.load('<path>/X_cleaned_attempt_new.npy',allow_pickle=True)
#Y = np.load('<path>/Y_cleaned_attempt_new.npy',allow_pickle=True)
#X = np.asarray(X).astype('float32')
#Y = np.asarray(Y).astype('float32')

# i'm using colab, so it's slightly more complicated
ytrainval_byteseq = files.upload() # choose 'Y_cleaned_attempt_new.npy' from local system
ytrainval_filelike = io.BytesIO(ytrainval_byteseq['Y_cleaned_attempt_new.npy']) # create file-like object
ytrainval = np.load(ytrainval_filelike,allow_pickle=True) # create regular numpy array
xtrainval_byteseq = files.upload() # choose 'X_cleaned_attempt_new.npy' from local system
xtrainval_filelike = io.BytesIO(xtrainval_byteseq['X_cleaned_attempt_new.npy']) # create file-like object
xtrainval = np.load(xtrainval_filelike,allow_pickle=True) # create regular numpy array
ytest_byteseq = files.upload() # choose 'Y_test.npy' from local system
ytest_filelike = io.BytesIO(ytest_byteseq['Y_test.npy']) # create file-like object
ytest = np.load(ytest_filelike,allow_pickle=True) # create regular numpy array
xtest_byteseq = files.upload() # choose 'X_test.npy' from local system
xtest_filelike = io.BytesIO(xtest_byteseq['X_test.npy']) # create file-like object
xtest = np.load(xtest_filelike,allow_pickle=True) # create regular numpy array

Saving Y_cleaned_attempt_new.npy to Y_cleaned_attempt_new.npy


Saving X_cleaned_attempt_new.npy to X_cleaned_attempt_new.npy


Saving Y_test.npy to Y_test.npy


Saving X_test.npy to X_test.npy


In [3]:
#check shapes
print(xtrainval.shape)
print(ytrainval.shape)
print(xtest.shape)
print(ytest.shape)

#ytrainval needs to be converted to number representation of labels
feature_map = {'Rock':0,'Electronic':1,'Experimental':2,'Hip-Hop':3,'Folk':4,'Instrumental':5,'Pop':6,
              'International':7,'Classical':8,'Old-Time / Historic':9, 'Jazz':10,'Country':11,'Soul-RnB':12,
              'Spoken':13,'Blues':14,'Easy Listening':15}
for i in range(ytrainval.shape[0]):
    ytrainval[i] = float(feature_map[ytrainval[i]])
ytrainval[i] = np.asarray(ytrainval[i]).astype('float32')

print(ytrainval)
print(ytest)

(9349, 520)
(9349,)
(40249, 520)
(40249,)
[3.0 3.0 3.0 ... 4.0 0.0 array(0., dtype=float32)]
[3.0 6.0 0.0 ... 0.0 0.0 array(0., dtype=float32)]


In [4]:
# for the preliminary report, i did a roughly 80-20 split, but used the 20% as the training set to improve speed
# so will recombine and do a 90-10 split, using the 90% as training/validation
xfull = np.concatenate((xtrainval,xtest),axis=0)
yfull = np.concatenate((ytrainval,ytest),axis=0)
print(xfull.shape)
print(yfull.shape)

(49598, 520)
(49598,)


In [28]:
# finish preprocessing

# ~50000 instances in full set, so could get away with 90-10 split probably
xtrainval, xtest, ytrainval, ytest = train_test_split(xfull,yfull,train_size=int(0.9*(xtest.shape[0]+xtrainval.shape[0])))

# split into training and validation
xtrain, xval, ytrain, yval = train_test_split(xtrainval,ytrainval,train_size=int(0.9*xtrainval.shape[0]))

# one-hot encoding
ytrain_1hot = to_categorical(ytrain)
yval_1hot = to_categorical(yval)
ytest_1hot = to_categorical(ytest)

print(ytrain_1hot.shape)
print(yval_1hot.shape)
print(ytest_1hot.shape)

(40174, 16)
(4464, 16)
(4960, 16)


In [6]:
'''
print(xtrain.shape)
print(xtrain[0,:])
print(xval[0,:])
print(xtest[0,:])
# starting with MLP; will need to un-flatten features into time-series signal for cnn implementation
# looks like data isn't normalized
'''

(40174, 520)
[-1.67627648e-01 -3.20516735e-01 -5.47658861e-01 -9.71182227e-01
 -1.76922217e-01 -4.82989877e-01  1.08642364e+00 -1.11493632e-01
 -4.72926460e-02 -3.23914438e-01 -1.07533276e-01 -3.57802957e-01
  6.31417334e-01  6.77803636e-01  6.32379293e-01  6.52357936e-01
  6.62765980e-01  5.71075618e-01  6.72195315e-01  6.65100396e-01
  6.97125554e-01  6.43173218e-01  6.86596274e-01  6.44778252e-01
  2.21735671e-01  2.48877287e-01  2.60894954e-01  2.69073159e-01
  2.11407930e-01  1.64966047e-01  1.69164568e-01  2.44937539e-01
  2.98581898e-01  2.98516273e-01  3.07941079e-01  2.41853774e-01
  2.25851804e-01  2.44530037e-01  2.65893698e-01  2.39126056e-01
  2.05803096e-01  1.64028451e-01  1.37677804e-01  2.32101724e-01
  2.78784692e-01  2.94118732e-01  2.93991119e-01  2.32563943e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.59775287

In [37]:
# will check feature list to understand how CNN might be implemented
# i don't think tf.keras is particularly well suited for this task, because 
# we would need several different layers for the time series of different
# dimensions.
# so maybe an ensemble CNN would be a good idea?
# i want to try that
chroma_cens = xtrain[:,:84] # length 12
chroma_cqt = xtrain[:,84:168] # length 12
chroma_stft = xtrain[:,168:252]# length 12
chroma_mfcc = xtrain[:,252:392] # length 20
rmse = xtrain[:,392:399]
spectral_bandwidth = xtrain[:,399:406]
spectral_centroid = xtrain[:,406:413]
spectral_contrast = xtrain[:,413:462]# length 7
spectral_rolloff = xtrain[:,462:469] 
tonnetz = xtrain[:,469:511] # length 6
zcr = xtrain[:,511:518]
dates = xtrain[:,518]
duration = xtrain[:,519]

print(chroma_cens.shape)

(40174, 84)


In [38]:
# currently, the features are a timeseries for each statistic
# but we want to reindex so that all statistics are intermingled
# but grouped by time
# this function will help with that
def get_index_order(subinterval_length):
    idxs = []
    for i in range(1,subinterval_length+1):
        idx = []
        for j in range(1,8):
            idx.append(subinterval_length*(j-1)+i-1)
        idxs.append(idx)
    idxs = np.array(idxs).flatten()
    #print(chroma_cens_idxs)
    return idxs

In [39]:
order_12 = get_index_order(12)
order_20 = get_index_order(20)
order_7 = get_index_order(7)
order_6 = get_index_order(6)
#print(order_12)
# reindex timeseries arrays
chroma_cens = chroma_cens[:,order_12] # length 12
chroma_cqt = chroma_cqt[:,order_12] # length 12
chroma_stft = chroma_stft[:,order_12] # length 20
chroma_mfcc = chroma_mfcc[:,order_20] # length 20
spectral_contrast = spectral_contrast[:,order_7] # length 7
tonnetz = tonnetz[:,order_6] # length 6

#reshape
chroma_cens = chroma_cens.reshape(xtrain.shape[0],int(chroma_cens.shape[1]/7),7)
chroma_cqt = chroma_cqt.reshape(xtrain.shape[0],int(chroma_cqt.shape[1]/7),7)
chroma_stft = chroma_stft.reshape(xtrain.shape[0],int(chroma_stft.shape[1]/7),7)
chroma_mfcc = chroma_mfcc.reshape(xtrain.shape[0],int(chroma_mfcc.shape[1]/7),7)
rmse = rmse.reshape(xtrain.shape[0],int(rmse.shape[1]/7),7)
spectral_bandwidth = spectral_bandwidth.reshape(xtrain.shape[0],int(spectral_bandwidth.shape[1]/7),7)
spectral_centroid = spectral_centroid.reshape(xtrain.shape[0],int(spectral_centroid.shape[1]/7),7)
spectral_contrast = spectral_contrast.reshape(xtrain.shape[0],int(spectral_contrast.shape[1]/7),7)
spectral_rolloff = spectral_rolloff.reshape(xtrain.shape[0],int(spectral_rolloff.shape[1]/7),7)
tonnetz = tonnetz.reshape(xtrain.shape[0],int(tonnetz.shape[1]/7),7)
zcr = zcr.reshape(xtrain.shape[0],int(zcr.shape[1]/7),7)

print(chroma_cens.shape)
#print(chroma_cens[0,:])

(40174, 12, 7)


In [40]:
print(xtrain.shape[0])
print(tonnetz.shape[0])

40174
40174


In [48]:
# define general structure
layer_list = [
          #layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(7,1,strides=1,kernel_initializer='lecun_normal',input_shape=(chroma_cens.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(8,1,strides=1,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), # should have 160 if length is 20?
          layers.BatchNormalization(),
          layers.Dropout(rate=0.2),
          layers.Dense(80,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)



In [43]:
# also need to split up xval 
xval_chroma_cens = xval[:,:84] # length 12
xval_chroma_cqt = xval[:,84:168] # length 12
xval_chroma_stft = xval[:,168:252]# length 12
xval_chroma_mfcc = xval[:,252:392] # length 20
xval_rmse = xval[:,392:399]
xval_spectral_bandwidth = xval[:,399:406]
xval_spectral_centroid = xval[:,406:413]
xval_spectral_contrast = xval[:,413:462] # length 7
xval_spectral_rolloff = xval[:,462:469] 
xval_tonnetz = xval[:,469:511] # length 6
xval_zcr = xval[:,511:518]
xval_dates = xval[:,518]
xval_duration = xval[:,519]

order_12 = get_index_order(12)
order_20 = get_index_order(20)
order_7 = get_index_order(7)
order_6 = get_index_order(6)

# reindex timeseries arrays
xval_chroma_cens = xval_chroma_cens[:,order_12] # length 12
xval_chroma_cqt = xval_chroma_cqt[:,order_12] # length 12
xval_chroma_stft = xval_chroma_stft[:,order_12] # length 12
xval_chroma_mfcc = xval_chroma_mfcc[:,order_20] # length 20
xval_spectral_contrast = xval_spectral_contrast[:,order_7] # length 7
xval_tonnetz = xval_tonnetz[:,order_6] # length 6

xval_chroma_cens = xval_chroma_cens.reshape(xval.shape[0],int(xval_chroma_cens.shape[1]/7),7)
xval_chroma_cqt = xval_chroma_cqt.reshape(xval.shape[0],int(xval_chroma_cqt.shape[1]/7),7)
xval_chroma_stft = xval_chroma_stft.reshape(xval.shape[0],int(xval_chroma_stft.shape[1]/7),7)
xval_chroma_mfcc = xval_chroma_mfcc.reshape(xval.shape[0],int(xval_chroma_mfcc.shape[1]/7),7)
xval_rmse = xval_rmse.reshape(xval.shape[0],int(xval_rmse.shape[1]/7),7)
xval_spectral_bandwidth = xval_spectral_bandwidth.reshape(xval.shape[0],int(xval_spectral_bandwidth.shape[1]/7),7)
xval_spectral_centroid = xval_spectral_centroid.reshape(xval.shape[0],int(xval_spectral_centroid.shape[1]/7),7)
xval_spectral_contrast = xval_spectral_contrast.reshape(xval.shape[0],int(xval_spectral_contrast.shape[1]/7),7)
xval_spectral_rolloff = xval_spectral_rolloff.reshape(xval.shape[0],int(xval_spectral_rolloff.shape[1]/7),7)
xval_tonnetz = xval_tonnetz.reshape(xval.shape[0],int(xval_tonnetz.shape[1]/7),7)
xval_zcr = xval_zcr.reshape(xval.shape[0],int(xval_zcr.shape[1]/7),7)

In [44]:
'''
print(xval_tonnetz.shape[0])
print(xval.shape[0])
print(yval_1hot.shape[0])
print(ytrain_1hot.shape[0])
print(chroma_cens.shape)
print(xtrain.shape)
'''

4464
4464
4464
40174
(40174, 12, 7)
(40174, 520)


In [26]:
'''
# let's just do some preliminary testing on this architecture for a few different things
#chroma_cens = chroma_cens.reshape(1,chroma_cens.shape[0],chroma_cens.shape[1])
#xval_chroma_cens = xval_chroma_cens.reshape(1,xval_chroma_cens.shape[0],xval_chroma_cens.shape[1])
#ytrain_1hot = ytrain_1hot.reshape(1,ytrain_1hot.shape[0],ytrain_1hot.shape[1])
#yval_1hot = yval_1hot.reshape(1,yval_1hot.shape[0],yval_1hot.shape[1])
chroma_cens_history = model.fit(chroma_cens_test, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_cens, yval_1hot))
chroma_cqt_history = model.fit(chroma_cqt, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_cqt, yval_1hot))
chroma_stft_history = model.fit(chroma_stft, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_stft, yval_1hot))
chroma_mfcc_history = model.fit(chroma_mfcc, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_mfcc, yval_1hot))
spectral_contrast_history = model.fit(spectral_contrast, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_spectral_contrast, yval_1hot))
tonnetz_history = model.fit(tonnetz, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_tonnetz, yval_1hot))
'''

ValueError: ignored

In [23]:
'''
#final preprocessing, hopefully: make our 7 filters
#chroma_cens = chroma_cens.reshape(chroma_cens.shape[0],int(chroma_cens.shape[1]/7),7)
xval_chroma_cens = xval_chroma_cens.reshape(xval_chroma_cens.shape[0],int(xval_chroma_cens.shape[1]/7),7)
# done above
'''

In [24]:
'''
print(chroma_cens.shape)
print(ytrain_1hot.shape)
print(xval_chroma_cens.shape)
print(yval_1hot.shape)
print(xval_chroma_cens.shape)
#print(chroma_cens_test.shape)
'''

(40174, 12, 7)
(40174, 16)
(4464, 12, 7)
(4464, 15)
(4464, 12, 7)


In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 1, 7)              56        
                                                                 
 batch_normalization_18 (Bat  (None, 1, 7)             28        
 chNormalization)                                                
                                                                 
 activation_15 (Activation)  (None, 1, 7)              0         
                                                                 
 conv1d_7 (Conv1D)           (None, 1, 8)              64        
                                                                 
 batch_normalization_19 (Bat  (None, 1, 8)             32        
 chNormalization)                                                
                                                                 
 activation_16 (Activation)  (None, 1, 8)             

In [50]:
# let's just do some preliminary testing on this architecture for a few different things
#chroma_cens = chroma_cens.reshape(1,chroma_cens.shape[0],chroma_cens.shape[1])
#xval_chroma_cens = xval_chroma_cens.reshape(1,xval_chroma_cens.shape[0],xval_chroma_cens.shape[1])
#ytrain_1hot = ytrain_1hot.reshape(1,ytrain_1hot.shape[0],ytrain_1hot.shape[1])
#yval_1hot = yval_1hot.reshape(1,yval_1hot.shape[0],yval_1hot.shape[1])
chroma_cens_history = model.fit(chroma_cens, ytrain_1hot, batch_size=32, epochs=100, validation_data=(xval_chroma_cens, yval_1hot))
#chroma_cqt_history = model.fit(chroma_cqt, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_cqt, yval_1hot))
#chroma_stft_history = model.fit(chroma_stft, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_stft, yval_1hot))
#chroma_mfcc_history = model.fit(chroma_mfcc, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_chroma_mfcc, yval_1hot))
#spectral_contrast_history = model.fit(spectral_contrast, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_spectral_contrast, yval_1hot))
#tonnetz_history = model.fit(tonnetz, ytrain_1hot, batch_size=32, epochs=10, validation_data=(xval_tonnetz, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 7s 6ms/step - loss: 1.6911 - accuracy: 0.4204 - val_loss: 1.7210 - val_accuracy: 0.4299
Epoch 2/100
1256/1256 [==============================] - 6s 4ms/step - loss: 1.6841 - accuracy: 0.4209 - val_loss: 2.0921 - val_accuracy: 0.3609
Epoch 3/100
1256/1256 [==============================] - 6s 4ms/step - loss: 1.6765 - accuracy: 0.4272 - val_loss: 1.6987 - val_accuracy: 0.4285
Epoch 4/100
1256/1256 [==============================] - 5s 4ms/step - loss: 1.6759 - accuracy: 0.4239 - val_loss: 1.9108 - val_accuracy: 0.4014
Epoch 5/100
1256/1256 [==============================] - 5s 4ms/step - loss: 1.6643 - accuracy: 0.4306 - val_loss: 1.6715 - val_accuracy: 0.4427
Epoch 6/100
1256/1256 [==============================] - 6s 4ms/step - loss: 1.6624 - accuracy: 0.4311 - val_loss: 1.9134 - val_accuracy: 0.4070
Epoch 7/100
1256/1256 [==============================] - 5s 4ms/step - loss: 1.6566 - accuracy: 0.4304 - val_loss: 1.9863 - val_ac

KeyboardInterrupt: ignored

In [51]:
layer_list = [
          #layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(100,2,strides=1,kernel_initializer='lecun_normal',padding='valid',input_shape=(chroma_cens.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), # should have 160 if length is 20?
          #layers.Dropout(rate=0.2),
          layers.Dense(80,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)


In [52]:
chroma_cens_history = model.fit(chroma_cens, ytrain_1hot, batch_size=32, epochs=100, validation_data=(xval_chroma_cens, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 8s 6ms/step - loss: 1.9884 - accuracy: 0.3347 - val_loss: 1.8796 - val_accuracy: 0.3600
Epoch 2/100
1256/1256 [==============================] - 7s 6ms/step - loss: 1.7759 - accuracy: 0.3915 - val_loss: 1.9931 - val_accuracy: 0.3190
Epoch 3/100
1256/1256 [==============================] - 7s 6ms/step - loss: 1.7249 - accuracy: 0.4128 - val_loss: 1.8280 - val_accuracy: 0.3676
Epoch 4/100
1256/1256 [==============================] - 7s 5ms/step - loss: 1.6911 - accuracy: 0.4243 - val_loss: 1.7441 - val_accuracy: 0.4084
Epoch 5/100
1256/1256 [==============================] - 7s 5ms/step - loss: 1.6650 - accuracy: 0.4341 - val_loss: 1.6858 - val_accuracy: 0.4384
Epoch 6/100
1256/1256 [==============================] - 7s 5ms/step - loss: 1.6432 - accuracy: 0.4396 - val_loss: 1.9749 - val_accuracy: 0.3190
Epoch 7/100
1256/1256 [==============================] - 7s 5ms/step - loss: 1.6252 - accuracy: 0.4466 - val_loss: 1.6977 - val_ac

In [55]:
'''
big_CNN_thing = np.concatenate((chroma_cens,chroma_cqt,chroma_stft,chroma_mfcc,rmse,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,tonnetz,zcr),axis=-1)
big_CNN_thing_val = np.concatenate((xval_chroma_cens,xval_chroma_cqt,xval_chroma_stft,xval_chroma_mfcc,xval_rmse,xval_spectral_centroid,xval_spectral_bandwidth,xval_spectral_contrast,xval_spectral_rolloff,xval_tonnetz,xval_zcr),axis=-1)
print(big_CNN_thing.shape)
print(big_CNN_thing_val.shape)
'''

ValueError: ignored

In [56]:
big_CNN_thing = np.concatenate((chroma_cens,chroma_cqt,chroma_stft,),axis=-1) # the things of length 12
big_CNN_thing_val = np.concatenate((xval_chroma_cens,xval_chroma_cqt,xval_chroma_stft),axis=-1)
print(big_CNN_thing.shape)
print(big_CNN_thing_val.shape)

(40174, 12, 21)
(4464, 12, 21)


In [63]:
layer_list = [
          layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(100,2,strides=1,kernel_initializer='lecun_normal',padding='valid',input_shape=(big_CNN_thing.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(200,2,strides=2,kernel_initializer='lecun_normal',padding='valid'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), 
          #layers.Dropout(rate=0.2),
          layers.Dense(80,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
big_CNN_thing_history = model.fit(big_CNN_thing, ytrain_1hot, batch_size=32, epochs=100, validation_data=(big_CNN_thing_val, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 13s 9ms/step - loss: 1.7036 - accuracy: 0.4587 - val_loss: 1.5511 - val_accuracy: 0.4888
Epoch 2/100
1256/1256 [==============================] - 11s 9ms/step - loss: 1.4834 - accuracy: 0.5089 - val_loss: 1.4736 - val_accuracy: 0.5343
Epoch 3/100
1256/1256 [==============================] - 11s 9ms/step - loss: 1.4328 - accuracy: 0.5277 - val_loss: 1.4685 - val_accuracy: 0.5284
Epoch 4/100
1256/1256 [==============================] - 11s 9ms/step - loss: 1.4027 - accuracy: 0.5392 - val_loss: 1.4386 - val_accuracy: 0.5347
Epoch 5/100
1256/1256 [==============================] - 11s 9ms/step - loss: 1.3739 - accuracy: 0.5472 - val_loss: 1.4188 - val_accuracy: 0.5361
Epoch 6/100
1256/1256 [==============================] - 11s 9ms/step - loss: 1.3510 - accuracy: 0.5545 - val_loss: 1.3897 - val_accuracy: 0.5533
Epoch 7/100
1256/1256 [==============================] - 11s 9ms/step - loss: 1.3344 - accuracy: 0.5627 - val_loss: 1.4160 -

KeyboardInterrupt: ignored

In [59]:
layer_list = [
          layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(100,2,strides=1,kernel_initializer='lecun_normal',padding='valid',input_shape=(big_CNN_thing.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(200,2,strides=2,kernel_initializer='lecun_normal',padding='valid'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), 
          #layers.Dropout(rate=0.2),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
big_CNN_thing_history = model.fit(big_CNN_thing, ytrain_1hot, batch_size=32, epochs=100, validation_data=(big_CNN_thing_val, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 11s 8ms/step - loss: 1.7215 - accuracy: 0.4427 - val_loss: 1.6500 - val_accuracy: 0.4648
Epoch 2/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.5603 - accuracy: 0.4889 - val_loss: 1.5976 - val_accuracy: 0.4810
Epoch 3/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.5066 - accuracy: 0.5046 - val_loss: 1.6085 - val_accuracy: 0.4899
Epoch 4/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.4772 - accuracy: 0.5179 - val_loss: 1.5603 - val_accuracy: 0.4960
Epoch 5/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.4466 - accuracy: 0.5254 - val_loss: 1.5180 - val_accuracy: 0.5184
Epoch 6/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.4204 - accuracy: 0.5351 - val_loss: 1.5125 - val_accuracy: 0.5164
Epoch 7/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.4012 - accuracy: 0.5388 - val_loss: 1.4894 - val_a

In [61]:
layer_list = [
          layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(100,2,strides=1,kernel_initializer='lecun_normal',padding='valid',input_shape=(big_CNN_thing.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(200,2,strides=2,kernel_initializer='lecun_normal',padding='valid',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), 
          #layers.Dropout(rate=0.2),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
big_CNN_thing_history = model.fit(big_CNN_thing, ytrain_1hot, batch_size=32, epochs=100, validation_data=(big_CNN_thing_val, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 10s 7ms/step - loss: 4.1487 - accuracy: 0.3982 - val_loss: 2.4770 - val_accuracy: 0.4091
Epoch 2/100
1256/1256 [==============================] - 9s 7ms/step - loss: 2.2771 - accuracy: 0.4353 - val_loss: 2.2098 - val_accuracy: 0.4294
Epoch 3/100
1256/1256 [==============================] - 9s 7ms/step - loss: 2.0673 - accuracy: 0.4605 - val_loss: 2.0449 - val_accuracy: 0.4550
Epoch 4/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.8540 - accuracy: 0.4811 - val_loss: 1.8079 - val_accuracy: 0.4816
Epoch 5/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.7119 - accuracy: 0.4956 - val_loss: 1.7469 - val_accuracy: 0.4866
Epoch 6/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.6574 - accuracy: 0.5009 - val_loss: 1.6933 - val_accuracy: 0.4973
Epoch 7/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.6324 - accuracy: 0.5031 - val_loss: 1.7339 - val_a

In [62]:
layer_list = [
          layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(100,2,strides=1,kernel_initializer='lecun_normal',padding='valid',input_shape=(big_CNN_thing.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(200,2,strides=2,kernel_initializer='lecun_normal',padding='valid',kernel_regularizer='l2'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), 
          #layers.Dropout(rate=0.2),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
big_CNN_thing_history = model.fit(big_CNN_thing, ytrain_1hot, batch_size=32, epochs=100, validation_data=(big_CNN_thing_val, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 10s 7ms/step - loss: 2.2731 - accuracy: 0.4327 - val_loss: 1.7559 - val_accuracy: 0.4668
Epoch 2/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.7070 - accuracy: 0.4647 - val_loss: 1.6731 - val_accuracy: 0.4756
Epoch 3/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.6414 - accuracy: 0.4784 - val_loss: 1.6901 - val_accuracy: 0.4671
Epoch 4/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.5894 - accuracy: 0.4990 - val_loss: 1.6226 - val_accuracy: 0.4913
Epoch 5/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.5435 - accuracy: 0.5089 - val_loss: 1.5678 - val_accuracy: 0.5123
Epoch 6/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.5048 - accuracy: 0.5199 - val_loss: 1.5854 - val_accuracy: 0.4955
Epoch 7/100
1256/1256 [==============================] - 9s 7ms/step - loss: 1.4823 - accuracy: 0.5257 - val_loss: 1.5304 - val_a

In [64]:
layer_list = [
          layers.BatchNormalization(),
          #layers.Conv1D(4,7,strides=7,kernel_initializer='lecun_normal',input_shape=(xtrain.shape[0],84)), #what if we just make 2nd dim xtrain.shape[1]?
          layers.Conv1D(100,2,strides=1,kernel_initializer='lecun_normal',padding='valid',input_shape=(chroma_cqt.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(200,2,strides=2,kernel_initializer='lecun_normal',padding='valid'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), 
          #layers.Dropout(rate=0.2),
          layers.Dense(80,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
big_CNN_thing_history = model.fit(chroma_cqt, ytrain_1hot, batch_size=32, epochs=100, validation_data=(xval_chroma_cqt, yval_1hot))

Epoch 1/100
1256/1256 [==============================] - 43s 8ms/step - loss: 1.8966 - accuracy: 0.3852 - val_loss: 1.7974 - val_accuracy: 0.4064
Epoch 2/100
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6900 - accuracy: 0.4311 - val_loss: 1.7884 - val_accuracy: 0.4113
Epoch 3/100
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6574 - accuracy: 0.4435 - val_loss: 1.9166 - val_accuracy: 0.4135
Epoch 4/100
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6266 - accuracy: 0.4548 - val_loss: 1.7732 - val_accuracy: 0.4245
Epoch 5/100
1256/1256 [==============================] - 10s 8ms/step - loss: 1.6072 - accuracy: 0.4622 - val_loss: 1.8081 - val_accuracy: 0.4265
Epoch 6/100
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5895 - accuracy: 0.4673 - val_loss: 1.8491 - val_accuracy: 0.4178
Epoch 7/100
1256/1256 [==============================] - 10s 8ms/step - loss: 1.5739 - accuracy: 0.4751 - val_loss: 1.8210 -

KeyboardInterrupt: ignored